# Particle Tracking with PRT

In [ ]:
from pathlib import Path

import flopy as fp
import numpy as np
import pandas as pd
import xarray as xr

import nlmod

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

In [ ]:
model_ws = Path("./scratch_model")
model_name = "from_scratch"

# create new subdir for PRT model
prt_ws = Path(model_ws) / "prt"

In [ ]:
ds = xr.open_dataset(model_ws / f"{model_name}.nc")

In [ ]:
# Create a new Simulation object
simprt = nlmod.sim.sim(ds, sim_ws=prt_ws)
tdis = nlmod.sim.tdis(ds, simprt)

# Add PRT model
prt = nlmod.prt.prt(ds, simprt)

# DIS: discretization package
dis = nlmod.prt.dis(ds, prt)

# MIP: matrix input package(?)
mip = nlmod.prt.mip(ds, prt, porosity=0.3)

# PRP: particle release package
pdata = fp.modpath.ParticleData(
    [
        (k, i, j)
        for i in np.arange(0, ds.sizes["y"], 11)
        for j in np.arange(0, ds.sizes["x"], 11)
        for k in [0, 1, 2]
    ],
    structured=True,
)
releasepts = list(pd.to_prp(prt.modelgrid, global_xy=False))
prp = nlmod.prt.prp(ds, prt, packagedata=releasepts, perioddata={0: ["FIRST"]})

# FMI: flow model interface
gwf_budget_file = Path(ds.model_ws) / f"{model_name}.cbc"
gwf_head_file = Path(ds.model_ws) / f"{model_name}.hds"
fmi = fp.mf6.ModflowPrtfmi(
    prt,
    packagedata=[
        ("GWFHEAD", gwf_head_file.absolute().resolve()),
        ("GWFBUDGET", gwf_budget_file.absolute().resolve()),
    ],
)

# OC: output control
oc = nlmod.prt.oc(ds, prt)

# EMS: explicit model solution
ems = nlmod.sim.ems(simprt, model=prt)

In [ ]:
simprt.write_simulation()
simprt.run_simulation()

Load pathline data

In [ ]:
pathlines = pd.read_csv(model_ws / "prt" / trackcsvfile_prt)

Plot results

In [ ]:
pmv = fp.plot.PlotMapView(prt)
# pmv.plot_grid(lw=0.5, color="k")
pmv.plot_pathline(
    pathlines,
    layer="all",
    colors="0.7",
    lw=1.0,
)
for ilay in [0, 1, 2]:
    mask = pathlines["ilay"] == (ilay + 1)
    pmv.plot_pathline(
        pathlines.loc[mask],
        layer=ilay + 1,
        # colors=f"C{ilay}",
        lw=0.0,
        marker=".",
        ms=4,
        markercolor=f"C{ilay}",
        markerevery=5,
    )
pmv.plot_endpoint(pathlines, color="k", marker=".", direction="starting", zorder=10)

# plot river
xyriv = [(250, -500), (300, -300), (275, 0), (200, 250), (175, 500)]
# plot well
pmv.ax.plot(
    [xy[0] for xy in xyriv],
    [xy[1] for xy in xyriv],
    "b-",
    lw=4,
    label="river",
)
pmv.ax.plot([100, 200], [-50, 150], "rs", ms=5)
handles = [
    pmv.ax.plot([], [], "C0.", ms=5, label="Layer 0")[0],
    pmv.ax.plot([], [], "C1.", ms=5, label="Layer 1")[0],
    pmv.ax.plot([], [], "C2.", ms=5, label="Layer 2")[0],
]
labels = [h.get_label() for h in handles]
pmv.ax.legend(handles, labels, loc=(0, 1), frameon=False, ncol=3)
pmv.ax.set_xlabel("X [m]")
pmv.ax.set_ylabel("Y [m]");